In [1]:
from icalendar import Calendar, Event
import requests
import pandas as pd
import random
from datetime import datetime, timedelta

def create_ics_events(events, filename="events.ics"):
    """
    Create an ICS file with multiple events

    Parameters:
    - events: list of dicts, each containing:
        - summary: str, event title
        - description: str, event description
        - start_time: datetime, event start time
        - end_time: datetime, event end time
        - location: str, event location (optional)
    - filename: str, output filename (optional)
    """
    # Create calendar
    cal = Calendar()

    # Add calendar metadata
    cal.add('prodid', '-//Vic Term Weeks Calendar//EN')
    cal.add('version', '2.0')
    cal.add('calscale', 'GREGORIAN')
    cal.add('method', 'PUBLISH')
    cal.add('x-wr-calname', 'Vic Term Weeks')
    cal.add('x-wr-timezone', 'Australia/Melbourne')

    # Add events
    for event_data in events:  # termlist contains lists of dicts
        event = Event()
        event.add('summary', event_data[0]['summary'])
        event.add('description', event_data[0]['description'])
        event.add('dtstart', event_data[0]['start_time'])
        event.add('dtend', event_data[0]['end_time'])
        if event_data[0].get('location'):
            event.add('location', event_data[0]['location'])

        cal.add_component(event)

    # Write to file
    with open(filename, 'wb') as f:
        f.write(cal.to_ical())

    return f"Events saved to {filename}"

In [ ]:
# Download the ICS file
url = 'https://www.vic.gov.au/sites/default/files/2021-12/Victorian-school-term-dates.ics'
response = requests.get(url)
cal = Calendar.from_ical(response.content)

# Create dictionary to store events
school_terms = {}

# Process each event
for component in cal.walk('VEVENT'):
    summary = component.get('summary')
    start = component.get('dtstart').dt
    end = component.get('dtend').dt
    
    # Only include events from Term 2 2025 onwards
    if start.strftime('%Y%m%d') >= '20250422':  # Term 2 2025 starts
        # Create unique key with year
        year = start.strftime('%Y')
        key = f"{year} {summary}"
        
        school_terms[key] = {
            'start': start.strftime('%Y%m%d'),
            'end': end.strftime('%Y%m%d')
        }


df = pd.DataFrame(school_terms.values(), index = school_terms.keys())
df = df.reset_index()
df['startdt'] = pd.to_datetime(df['start'], format='%Y%m%d')
df['enddt'] = pd.to_datetime(df['end'], format='%Y%m%d')
df['Year'] = df['index'].apply(lambda row: row.split(' ')[0]) # Dervie year
df['Term'] = df['index'].apply(lambda row: " ".join(row.split(' ')[1:3])) # Dervie year
df['Pos']  = df['index'].apply(lambda row: row.split(' ')[3]) # Dervie start or end of term
df = df[~df['index'].str.endswith('schools)')].copy() # Ignore multiple term 1 start dates
df = df.pivot(index=['Year','Term'], columns='Pos', values='startdt').reset_index() # Transform data into terms

In [ ]:
termlist = []
for _, term in df.iterrows(): # For each term
    currday = term['starts']  # Start of term
    lastday = term['ends']    # end of term
    
    weeknum = 1 # Starting week

    while currday <= lastday: # While it is currently a term
        lastdayofweek = currday+ pd.Timedelta(days= 4 - currday.weekday()) # Find Friday
        assert lastdayofweek.weekday() == 4
        termlist.append([{'summary': f'{term["Term"]} Week {weeknum}', # Add the week
                         'description':  f'{term["Term"]} Week {weeknum}',
                             'start_time': currday,
                         'end_time' : lastdayofweek+pd.Timedelta(days=1)}])
        weeknum = weeknum + 1
        currday = lastdayofweek+pd.Timedelta(days=3) # Move to Monday (next week)


In [ ]:
"""
Lets sneak in some little bonus events.
"""

# Define the start and end dates
start_date = datetime(2025, 8, 1)
end_date = datetime(2030, 12, 1)

random_dates = [
    start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    for _ in range(5) # 5 random days
]
for cakeday in random_dates:
    print(cakeday)
    termlist.append([{'summary': 'Buy Anthony Cake',
                         'description':  'Anthonys favitour cake is red velvet',
                         'start_time': cakeday,
                         'end_time' : cakeday}])

2030-02-17 00:00:00
2029-03-26 00:00:00
2028-06-26 00:00:00
2030-04-05 00:00:00
2030-07-01 00:00:00


In [ ]:
create_ics_events(termlist, filename="TermDates.ics")

'Events saved to TermDates____.ics'